# Discrete Dynamical Systems with Wolfram Language!

Dynamical systems are rules describing the evolution of some variable $\vec{u}$ in time, according to some equations of motion given by a function $\vec{f}$.  
Here, we'll investigate _discrete dynamical systems_, often called _maps_, whereby time is discrete (in 'time-steps') and the equations of motion take the general form:

$$
\vec{u}_{n+1} = \vec{f}(\vec{u}_n,p,n)
$$

We'll use the Wolfram Language's `Compile` capabilities to compare our results with julia.

### De Jong attractor
Let's have a look at the following attractor:
$$
\begin{align}
x_{n+1} &= sin(a y_n) - cos(b x_n) \\
y_{n+1} &= sin(c x_n) - cos(d y_n)
\end{align}
$$

we define a function to perform this map

In [ ]:
naiveDeJong[{x_, y_}, {a_, b_, c_, d_}] :=
 {Sin[a y] - Cos[b x], 
  Sin[c x] - Cos[d y]}

Next we apply our map to an initial state, using our parameters:

In [ ]:
naiveDeJong[{1., 1.}, {-2, -2, -1.2, 2}]

We can then `Nest` this iterator easily using `NestList`:

In [ ]:
Graphics[Point[
  NestList[naiveDeJong[#, {-2, -2, -1.2, 2}] &, {1., 1.}, 
   100000]]]

## Visualization
Post-processing binning is similarly very straightforward using the high-level `BinCounts` function:

In [ ]:
With[{bins=BinCounts[
 NestList[naiveDeJong[#, {-2, -2, -1.2, 2}] &, {1., 1.}, 
  10^6], {-2, 2, 0.005}, {-2, 2, 0.005}]},
  ArrayPlot[Log[bins+1]]]

## Compiled Binning
In general, the above is quite slow - and will run into memory issues since we need to store all the points before binning them.  
Instead, we can `Compile` down to C. We might as-well bin along the way:

In [ ]:
dejongCompiled = 
 Compile[{{xmin, _Real}, {xmax, _Real}, {ymin, _Real}, {ymax, _Real}, 
  {delta, _Real}, {itmax, _Integer}, {a, _Real, 0}, {b, _Real, 0}, {c, _Real, 0}, {d, _Real, 0}}, 
  
  Block[{bins, dimx, dimy, x, y, tx, ty}, 
   bins =Table[0, {Floor[(xmax - xmin)/delta] + 1}, {Floor[(ymax - ymin)/delta] + 1}];
   {dimx, dimy} = Dimensions[bins];
   
   {x, y} = {0., 0.};
   Do[{x, y} = {Sin[a y] - Cos[b x], Sin[c x] - Cos[d y]};
    
    tx = Floor[(x - xmin)/delta] + 1;
    ty = Floor[(y - ymin)/delta] + 1;
    
    If[tx >= 1 && tx <= dimx && ty >= 1 && ty <= dimy, 
     bins[[tx, ty]] += 1], {i, 1, itmax}];
   
   bins], CompilationTarget -> "C", RuntimeOptions -> "Speed"];

In [ ]:
RepeatedTiming[bins=dejongCompiled[-2., 2., -2., 2., 0.005, 10000000, -2.0, -2.0, -1.2, 2.0];]
ArrayPlot[
 Log[bins + 1], Frame -> False, ImageSize -> 500, 
 ColorFunction -> ColorData[{"SunsetColors","Reversed"}]]